# Multi-shot ASP solving with clingo in Python

This is an example to show how you can do multi-shot Answer Set Programming with [clingo](https://potassco.org/clingo/) in Python.

In [1]:
import clingo

## Multi-shot ASP solving

[*Multi-shot ASP solving*](https://arxiv.org/pdf/1705.09811.pdf) refers to the process of calling the ASP solver (e.g., clingo) on an answer set program, getting an answer (an answer set, or the conclusion that there are no answer sets), and then changing some parts of the program and having the solver continue its search—not from scratch, but continuing where it had stopped (e.g., keeping the learned nogoods), and so not repeating the same search as it had done the first time.


## Example using the wolf, goat & cabbage problem

To illustrate how this works, let's solve the [wolf, goat & cabbage problem](https://en.wikipedia.org/wiki/Wolf,_goat_and_cabbage_problem) using multi-shot ASP solving.

The task is to find the minimum number of moves to get the cabbage, the goat and the wolf safely to the other side, without anyone or anything getting eaten. If we don't know any bounds on this minimum number of moves, what we can do is first try it with 0 moves, if that doesn't work try it with 1 move, if that doesn't work try it with 2 moves, and so on, until we find the first number of moves for which there is a solution.

We could do this by calling clingo from scratch for each number `t` of moves. The disadvantage is that clingo will then do a lot of extra work. For example, all the intermediate information (which combinations of atoms lead to a conflict, etc) that clingo learned in the solving process for `t` would then be thrown away, and for `t+1` clingo would have to do all this work again.

So instead, let's use multi-shot solving, where we add new rules and facts to the program when `t` increases, allowing clingo to keep the intermediate information that it found in the previous steps.

To do this, we split our ASP program into two parts:
- `asp_code_base`, that is independent of the time steps, and only refers to time step `t=1`;
- `asp_code_step`, that we will use for each subsequent value of `t`.

In our case, the 'base' part of the program is as follows. We add the facts and rules of our program that are either independent of the time steps, or refer to time step 1.

In [2]:
asp_code_base = """
    % There are two sides of the river
    side(left;right).
    otherside(left,right).
    otherside(right,left).

    % There are three 'items'
    item(wolf;goat;cabbage).

    % These are the possible actions that we can take
    action(transfer(I)) :- item(I).
    action(empty_transfer).
    
    % Declare the first time step
    time(1).
    
    % Declare what is the case at time step 1
    at(1,I,left) :- item(I).
    at(1,boat,left).
"""

The 'step' part of the program is as follows. We add the facts and rules of our program that work for time step `t>1`, assuming that we have already added everything for previous time steps `t' < t`.

In [3]:
asp_code_step = """
    % Add the time step t
    time(t).

    % Now that t is the latest time step, we want to perform an action at time t-1
    1 { do(A,t-1) : action(A) } 1 :- time(t-1).
    
    % Require that the action at t-1 should be possible
    :- do(transfer(I),t-1), side(S), at(t-1,boat,S), not at(t-1,I,S).
    
    % Define where the items are at time t
    at(t,I,S) :- item(I), at(t-1,I,S), not do(transfer(I),t-1).
    at(t,I,S2) :- item(I), at(t-1,I,S1), do(transfer(I),t-1), otherside(S1,S2).
    
    % Define where the boat is at time t
    at(t,boat,left) :- t \ 2 = 1.
    at(t,boat,right) :- t \ 2 = 0.
    
    % Make sure nothing gets eaten at time t
    :- time(t), at(t,boat,S1), otherside(S1,S2), at(t,wolf,S2), at(t,goat,S2).
    :- time(t), at(t,boat,S1), otherside(S1,S2), at(t,cabbage,S2), at(t,goat,S2).

    % Define under what conditions we reached the goal at time t
    goal_reached(t) :- at(t,boat,right), at(t,I,right) : item(I).
    
    % We will use an external atom is_latest_time_step(t) to require that we reach the goal at time t
    % (an external atom so that we can 'turn off' this requirement in the future)
    #external is_latest_time_step(t).

    % Test whether the goal is reached
    % (only when is_latest_time_step(t) is true)
    :- is_latest_time_step(t), not goal_reached(t).
"""

What we have added to the 'step' part of the program so far does not yet require that we actually reach the goal at time step `t`. In order to do this, we will add a so-called *external* atom, let's call it `is_latest_time_step(t)`. This is an atom whose truth value we can later switch using the clingo API, so that we can turn it on when we want to require that the goal is reached at time step `t`, and turn it off when we move to `t+1` and onwards and want to let go of the requirement that we reach the goal at time step `t`.

Let's print any solutions in a human-readable format, using the following function.

In [4]:
def pretty_print(answer_set):
    
    # Use a dictionary to store the plan
    plan = dict()
    
    # Find all atoms with do/2 in the answer set
    # and store the corresponding action in the plan
    for atom in answer_set.symbols(atoms=True):
        if atom.name == "do":
            t = atom.arguments[1].number
            action = str(atom.arguments[0])
            plan[t] = action
    
    #
    time_steps = list(plan)
    time_steps.sort()
    for time in time_steps:
        print(f" {time}: {plan[time]}")

Finally, let's get to how we should call clingo in Python to make it all work.

In [5]:
# Pick a starting point and an upper bound for t
starting_step = 4
max_step = 25 # pro tip: use math.inf for no upper bound

# Set up clingo
control = clingo.Control()
control.configuration.solve.models = 1 # number of solutions to find

# Add the various parts of the program
# and indicate that the argument for step is called "t"
control.add("base", [], asp_code_base)
control.add("step", ["t"], asp_code_step)

# Ground the base part of the program
parts = []
parts.append(("base",[]))
control.ground(parts)
print("Grounding the base program..")

# Ground the step part of the program, for all t < starting step
for step in range(2, starting_step):
    parts = []
    parts.append(("step", [clingo.symbol.Number(step)]))
    control.ground(parts)
    print(f"Grounding the step program for t={step}..")

# Solving loop: starting with t=starting_step and going up by 1 each time
step = starting_step
found_solution = False

print()

while step <= max_step:
    print(f"Trying for t={step}")
    
    # Set any previous external atom "is_latest_time_step(t)" to false for t=step-1.
    control.release_external(clingo.Function("is_latest_time_step", [clingo.Number(step-1)]))
    
    # Add the step part of the program for the new value of t
    parts = []
    parts.append(("step", [clingo.symbol.Number(step)]))
    control.ground(parts)
    
    # Set the external atom "is_latest_time_step(t)" to true for t=step.
    control.assign_external(clingo.Function("is_latest_time_step", [clingo.Number(step)]), True)
    
    # Call clingo
    with control.solve(yield_=True) as handle:
        
        # Print any solutions that we may have found
        for answer_set in handle:
            found_solution = True
            print("Found a solution!")
            pretty_print(answer_set)

        # If we found a solution, call it a day!
        if found_solution:
            break

        # Else go to the next value of t and keep going..
        print("No solution yet..\n")
        step += 1

Grounding the base program..
Grounding the step program for t=2..
Grounding the step program for t=3..

Trying for t=4
No solution yet..

Trying for t=5
No solution yet..

Trying for t=6
No solution yet..

Trying for t=7
No solution yet..

Trying for t=8
Found a solution!
 1: transfer(goat)
 2: empty_transfer
 3: transfer(wolf)
 4: transfer(goat)
 5: transfer(cabbage)
 6: empty_transfer
 7: transfer(goat)
